In [0]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [0]:
cuda = True if torch.cuda.is_available() else False

In [0]:
cuda

True

In [0]:
# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(28), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=64,
    shuffle=True,
)

In [0]:
img_shape = (1, 28, 28)

In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [0]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

In [0]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [0]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()


In [0]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [0]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [0]:
for epoch in range(200):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(generator(z)), valid)

        g_loss.backward() 
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()
        
        if i % 100 == 0:
          print(
              "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
              % (epoch, 200, i, len(dataloader), d_loss.item(), g_loss.item())
          )

[Epoch 0/200] [Batch 0/938] [D loss: 1.113206] [G loss: 0.537843]
[Epoch 0/200] [Batch 100/938] [D loss: 0.485181] [G loss: 1.620304]
[Epoch 0/200] [Batch 200/938] [D loss: 0.726248] [G loss: 1.015321]
[Epoch 0/200] [Batch 300/938] [D loss: 0.629153] [G loss: 2.613583]
[Epoch 0/200] [Batch 400/938] [D loss: 0.385084] [G loss: 3.053779]
[Epoch 0/200] [Batch 500/938] [D loss: 0.464097] [G loss: 1.703355]
[Epoch 0/200] [Batch 600/938] [D loss: 0.728814] [G loss: 0.972529]
[Epoch 0/200] [Batch 700/938] [D loss: 0.445054] [G loss: 1.727736]
[Epoch 0/200] [Batch 800/938] [D loss: 0.650251] [G loss: 1.028201]
[Epoch 0/200] [Batch 900/938] [D loss: 0.503730] [G loss: 1.390272]
[Epoch 1/200] [Batch 0/938] [D loss: 0.367410] [G loss: 1.690794]
[Epoch 1/200] [Batch 100/938] [D loss: 0.518393] [G loss: 1.392962]
[Epoch 1/200] [Batch 200/938] [D loss: 0.617249] [G loss: 2.204370]
[Epoch 1/200] [Batch 300/938] [D loss: 0.496911] [G loss: 3.354573]
[Epoch 1/200] [Batch 400/938] [D loss: 0.636813] [G 